In [3]:
import pandas as pd
import numpy as np
import feather
import feather
import json
import re
import nltk
import unicodedata

from collections import Counter
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer


# Read data file into a pandas dataframe
read_df = feather.read_dataframe('../parsed_data/filtered_tip_data.feather', 'rb')

In [5]:
test = read_df.head(300)
# type(bus_df.stars[1])

In [10]:
# Helper functions for normalising text data

# Convert all words to lowercase, remove punctuation, tokenise and stem
# and remove stopwords, threshold = 10%
def norm_corpus(document):
    
    # unicode decode
    document = document.decode('utf-8')
    
    # lowercase and remove symbols
    tokenizer = RegexpTokenizer(r'\w+')
    doc_tokens = tokenizer.tokenize(document.lower())
        
    # remove stopwords
    doc_tokens = [word for word in doc_tokens if word not in stopwords.words('english')]
        
    # stem words
    stemmer = SnowballStemmer("english")
    doc_stem = [stemmer.stem(word) for word in doc_tokens]
        
    # make tokenised text one string
    norm_doc = " ".join(doc_stem)
    
    return norm_doc


# Vectorise keywords from normalised text to vector including only nouns and adjectives
def review_vector(norm_doc):

    # select all words categorised as nouns or adjectives
    # loop through each string i.e. review in the df column
    review_keyword_list = []
    doc = nltk.word_tokenize(norm_doc)

    # create tuple for each word in list: (word, tag)
    token_category = nltk.pos_tag(doc)  

    for word, tag in token_category:   
            
        # nouns
        if (tag == 'NN' or tag == 'NNS' or tag == 'NNP' or tag == 'NNPS'):
            review_keyword_list.append(word)
                
        # adjectives
        elif (tag == 'JJ' or tag == 'JJS' or tag == 'JJP' or tag == 'JJPS'):
            review_keyword_list.append(word)
        
        else:
            pass     
        
    review_keywords = " ".join(review_keyword_list)
        
    # vectorise string
    WORD = re.compile(r'\w+')
    review_vector = Counter(WORD.findall(review_keywords))
    
    
    return review_vector

In [8]:
test.columns.tolist()

['business_id',
 'date',
 'likes',
 'text',
 'type',
 'user_id',
 'latitude',
 'longitude',
 'name',
 'city',
 'stars',
 'review_count',
 'food_drink']

In [11]:
## Tip data
output_df = test.ix[:,['business_id', 'date', 'likes', 'text', 'type',
                       'user_id', 'latitude', 'longitude', 'name', 'city',
                       'stars', 'review_count', 'food_drink']]
output_df.text = test.text.apply(lambda x: norm_corpus(x))
print "tip text normalised, next: vectorise"
output_df.text = output_df.text.apply(lambda x: review_vector(x))

## Review data
# bus_df = bus_df[bus_df.stars > 3]
# output_df = bus_df.ix[:,['business_id', 'user_id', 'date', 'stars', 'votes', 'text']]
# output_df.text = output_df.text.apply(lambda x: norm_corpus(x))
# print "tip text normalised, next: vectorise"
# output_df.text = output_df.text.apply(lambda x: review_vector(x))
# output_df.head()

tip text normalised, next: vectorise


In [9]:
output_df.head()

,business_id,user_id,date,text
date,,,,
2013-04-18,cE27W9VPgO88Qxe4ol6y_g,-6rEfobYjMxpUWLNxszaxQ,2013-04-18,"{u'wast': 1, u'time': 1}"
2013-01-06,mVHrayjG3uZ_RLHkLj-AMg,EZ0r9dKKtEGVx2CdnowPCw,2013-01-06,"{u'depart': 1, u'polic': 1, u'rankin': 1, u'di..."
2013-12-03,KayYbHCt-RkbGcPdGOThNg,xb6zEQCw9I-Gl0g06e1KsQ,2013-12-03,"{u'great': 1, u'special': 1}"
2015-07-08,KayYbHCt-RkbGcPdGOThNg,QawZN4PSW7ng_9SP7pjsVQ,2015-07-08,"{u'great': 1, u'good': 1, u'food': 1, u'beer':..."
2015-10-25,1_lU0-eSWJCRvNGk78Zh9Q,MLQre1nvUtW-RqMTc4iC9A,2015-10-25,"{u'beauti': 1, u'restor': 1}"


In [6]:
## Group by same month in year to count number of emails per topic
## i.e. -> emails sent per topic monthly
# make date column index
output_df.index = pd.to_datetime(output_df.date, 
                                 format='%m/%d/%Y')

# perform groupby, summinng up dummies for count
monthly = pd.DataFrame.groupby(output_df, 
                               by=[output_df.index.year, 
                                   output_df.index.month]).aggregate(np.sum)